In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
import pprint
from uszipcode import SearchEngine
from config import api_key
from census import Census

c:\Users\alici\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# Read in data from csv file
crime_df = pd.read_csv("chicago_crimes_all_cleaned.csv")

# Remove District 31 as it has a very low count
crime_df = crime_df[crime_df["District"] != 31]
crime_df = crime_df[["Case Number", "Date", "Block", "Primary Type", "Description", "District", "Ward",
                                     "Year", "Latitude", "Longitude", "Location", "Population", "Zipcode"]]
crime_df

,Case Number,Date,Block,Primary Type,Description,District,Ward,Year,Latitude,Longitude,Location,Population,Zipcode
0,JC104662,12/31/2018 23:59,112XX S SACRAMENTO AVE,MINOR PROPERTY CRIME,TO VEHICLE,22,19,2018,41.689079,-87.696064,"(41.689078832, -87.696064026)",28804,60655
1,JC100043,12/31/2018 23:57,084XX S SANGAMON ST,MINOR PROPERTY CRIME,TO PROPERTY,6,21,2018,41.740521,-87.647391,"(41.740520866, -87.647390719)",68096,60620
2,JC100006,12/31/2018 23:56,018XX S ALLPORT ST,ASSAULT & BATTERY,AGG: HANDS/FIST/FEET NO/MINOR INJURY,12,25,2018,41.857068,-87.657625,"(41.857068095, -87.657625201)",79205,60608
3,JC100031,12/31/2018 23:55,078XX S SANGAMON ST,ASSAULT & BATTERY,DOMESTIC BATTERY SIMPLE,6,17,2018,41.751914,-87.647717,"(41.75191443, -87.647716532)",68096,60620
4,JC100026,12/31/2018 23:49,052XX W GLADYS AVE,ASSAULT & BATTERY,AGGRAVATED - HANDGUN,15,29,2018,41.875684,-87.760479,"(41.87568438, -87.760479356)",47712,60644
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,JF371972,8/27/2022 2:30,023XX S HAMLIN AVE,THEFT (ALL TYPES),AUTOMOBILE,10,22,2022,41.849272,-87.719809,"(41.849272146, -87.719809208)",77352,60623
1048571,JF376352,7/20/2022 12:02,082XX S EAST END AVE,THEFT (ALL TYPES),OVER $500,4,8,2022,41.745362,-87.582937,"(41.745362374, -87.582937407)",48973,60649
1048572,JF375554,8/29/2022 22:20,007XX W 109TH ST,MINOR PROPERTY CRIME,TO VEHICLE,22,34,2022,41.696007,-87.641705,"(41.696006842, -87.641705115)",49796,60643
1048573,JF375348,8/29/2022 18:45,002XX W 26TH ST,THEFT (ALL TYPES),"CYCLE, SCOOTER, BIKE WITH VIN",9,11,2022,41.845498,-87.633792,"(41.845498317, -87.633792155)",53085,60616


In [3]:
crime_df.to_csv("chicago_crimes_all_cleaned_final.csv", index = False)

In [4]:
# Create a SearchEngine object 
search = SearchEngine()

In [5]:
# Define a function to get zip code from latitude and longitude
def fill_zip_code(row):
    # Use uszipcode's search by coordinates function, and set a small radius since we're only looking at Chicago
    result = search.by_coordinates(row['Latitude'], row['Longitude'], radius = 5)
    # If the result of the search is not empty
    if result:
        # Return the first element (zipcode) 
        return result[0].zipcode
    else:
        # If the result doesn't have a zipcode to pull, don't return anything
        return None

In [6]:
# Run the latitude and longitude through the function to get the zip code. Note: this takes an extremely long amount of time.
crime_df["Zipcode"] = crime_df.apply(fill_zip_code, axis=1)



In [ ]:
# Check to make sure zip codes were filled
crime_df

In [ ]:
# Change Zip code from an object to a float for merging purposes
crime_df["Zipcode"] = crime_df["Zipcode"].astype(float)

In [ ]:
# Save the file to .csv
crime_df.to_csv("zip_codes_added", index = False)

In [ ]:
# Check the amount of null values in the zip code column
print(f"The number of null values is: {crime_df.isna().sum()}")

In [ ]:
# Read in "zip_codes_added" to not run the crime_df["Zipcode"] above again. **Run from here to save time**
crime_df = pd.read_csv("Resources/zip_codes_added")

In [ ]:
# Read in the census data for 2018-2021. Note that we will have to get the 2022 census popultation data from another source (the census API)
pop_counts = pd.read_csv("Resources/Chicago_Population_Counts.csv")
pop_counts

In [ ]:
# Check to see how many null values we have
pop_counts.isna().sum()

In [ ]:
# Merge crime DataFrame with population counts by zipcode
merged_df = crime_df.merge(pop_counts, on = ["Zipcode", "Year"], how = "left")
merged_df

In [ ]:
# See what columns we have
merged_df.columns

In [ ]:
# Clean columns to the ones we want
crime_cleaned_df = merged_df[["Case Number", "Date", "Block", "Primary Type", "Description", "District", "Ward",
                                     "Year", "Latitude", "Longitude", "Location", "Population - Total", "Zipcode"]]
crime_cleaned_df

In [ ]:
# Change the name of "Population - Total" so it's easier to merge
crime_cleaned_df = crime_cleaned_df.rename(columns={"Population - Total" : "Population"})
crime_cleaned_df

In [ ]:
# Check the amount of null values in the merged data frame.  
print(f"The number of null values is: {crime_cleaned_df.isna().sum()}")

In [ ]:
# See the year the null values are coming from
null_population = crime_cleaned_df.loc[crime_cleaned_df["Population"].isnull()]
null_population

In [ ]:
# Count the amount of null values per year. Note we have not added population for 2022 yet
year_null = null_population["Year"].value_counts()
year_null

In [ ]:
# Check the data types of the merged data frame
col_types = crime_cleaned_df.dtypes
print(col_types)

In [ ]:
# Create an instance of the Census library
c = Census(
    api_key,
    year = 2021
)


In [ ]:
# Run Census Search to retrieve population data on all zip codes (ACS5 Census). We can then match them up to the crime table later.
census_data = c.acs5.get(
    (
        "NAME",
        # Estimate of the total population
        "B01003_001E"
    ),
    {'for': 'zip code tabulation area:*'}
)

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column renaming
census_pd = census_pd.rename(
    columns = {
        "B01003_001E": "Population",
        "zip code tabulation area": "Zipcode"
    }
)

# Configure the final DataFrame
census_pd = census_pd[
    [
        "Population",
        "Zipcode" 
    ]
]

# Display DataFrame length and sample data
print(f"Number of rows in the DataFrame: {len(census_pd)}")
census_pd.head()

In [ ]:
# Change zipcode from object to integer
census_pd["Zipcode"] = census_pd["Zipcode"].astype(int)


In [ ]:
# This fills in filling in missing null values with the census data
cen_merge_df = pd.merge(crime_cleaned_df[crime_cleaned_df['Population'].isnull()], census_pd, on='Zipcode', how = "left")
cen_merge_df

In [ ]:
# Rename the column for population and drop unused rows
cen_cleaned_df = cen_merge_df[["Case Number", "Date", "Block", "Primary Type", "Description", "District", "Ward",
                                     "Year", "Latitude", "Longitude", "Location", "Population_y", "Zipcode"]]
cen_cleaned_df = cen_cleaned_df.rename(columns = {"Population_y" : "Population"})
cen_cleaned_df

In [ ]:
# Concatenate cen_cleaned_df and crime_cleaned_df to get the final dataset
final_df = pd.concat([crime_cleaned_df, cen_cleaned_df])
final_df

In [ ]:
# Drop any remaining null rows
final_df = final_df.dropna(how="any")
final_df

In [ ]:
# Make sure zip codes are in Chicago
sort_zip = final_df["Zipcode"].sort_values().unique()
sort_zip

In [ ]:
# This was checked with the Name Census
# List of 67 ZIP codes in Chicago, Illinois. Name Census. (n.d.). https://namecensus.com/zip-codes/illinois/city/chicago/ 
non_chicago_zips = [
    46320,
    60077,
    60171,
    60176,
    60301,
    60302,
    60304,
    60406,
    60419,
    60453,
    60456, 
    60459, 
    60501,
    60534, 
    60712,
    60803,
    60805,
    65692]
         

In [ ]:
# Remove the zip codes that are outside of Chicago
final_df = final_df[~final_df["Zipcode"].isin(non_chicago_zips)]
final_df


In [ ]:
final_df.to_csv("chicago_crimes_all_cleaned.csv", index = False)